# DNN

In [42]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Carga los datos
data = pd.read_csv('../../data/partidos_limpio.csv')

# Preparar las etiquetas binarias para los resultados de los partidos
data['Winner'] = (data['Score'].apply(lambda x: int(x.split('-')[0]) > int(x.split('-')[1]))).astype(int)

# Selección de características y datos categóricos para codificación
features = data[['Home', 'Away', 'Referee']]
target = data['Winner']

# Preprocesamiento con OneHotEncoder para los datos categóricos
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ['Home', 'Away', 'Referee'])
])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Transformar los datos con el preprocesador
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Modelo de red neuronal
model = Sequential([
    Dense(128, activation='relu', input_shape=[X_train_transformed.shape[1]]),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compilación del modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
model.fit(X_train_transformed, y_train, epochs=50, batch_size=10)

# Evaluación del modelo en el conjunto de prueba
test_accuracy = model.evaluate(X_test_transformed, y_test)[1]
print("Precisión en el conjunto de prueba:", test_accuracy)


Epoch 1/50
46/46 [==============================] - 1s 1ms/step - loss: 0.6955 - accuracy: 0.4728
Epoch 2/50
46/46 [==============================] - 0s 1ms/step - loss: 0.6689 - accuracy: 0.7168
Epoch 3/50
46/46 [==============================] - 0s 1ms/step - loss: 0.6204 - accuracy: 0.7669
Epoch 4/50
46/46 [==============================] - 0s 2ms/step - loss: 0.5164 - accuracy: 0.7974
Epoch 5/50
46/46 [==============================] - 0s 947us/step - loss: 0.3963 - accuracy: 0.8322
Epoch 6/50
46/46 [==============================] - 0s 869us/step - loss: 0.2704 - accuracy: 0.9129
Epoch 7/50
46/46 [==============================] - 0s 840us/step - loss: 0.1690 - accuracy: 0.9630
Epoch 8/50
46/46 [==============================] - 0s 2ms/step - loss: 0.1107 - accuracy: 0.9739
Epoch 9/50
46/46 [==============================] - 0s 867us/step - loss: 0.0620 - accuracy: 0.9847
Epoch 10/50
46/46 [==============================] - 0s 889us/step - loss: 0.0441 - accuracy: 0.9891
Epoch 11/

In [43]:
# Cargar y revisar el contenido del archivo actualizado para la temporada 2023-2024
updated_data_path = '../../data/partidos_2023-2024_limpio.csv'
updated_data = pd.read_csv(updated_data_path)
updated_data.head()

,Season,Round,Day,Date,Home,Score,Away,Venue,Referee
0,2023-2024,Round of 16,Tue,2024-02-13,RB Leipzig,0-1,Real Madrid,Red Bull Arena,Irfan Peljto
1,2023-2024,Round of 16,Tue,2024-02-13,FC Copenhagen,1-3,Manchester City,Parken,José Sánchez
2,2023-2024,Round of 16,Wed,2024-02-14,Paris S-G,2-0,Real Sociedad,Parc des Princes,Marco Guida
3,2023-2024,Round of 16,Wed,2024-02-14,Lazio,1-0,Bayern Munich,Stadio Olimpico,François Letexier
4,2023-2024,Round of 16,Tue,2024-02-20,PSV Eindhoven,1-1,Dortmund,Philips Stadion,Srđan Jovanović


In [44]:
# Filtrar partidos sin resultado en el score
upcoming_matches = updated_data[updated_data['Score'].isna()]

# Mostrar los partidos próximos para verificar los datos
upcoming_matches


,Season,Round,Day,Date,Home,Score,Away,Venue,Referee
24,2023-2024,Semi-finals,Tue,2024-04-30,Bayern Munich,NaN,Real Madrid,Allianz Arena,NaN
25,2023-2024,Semi-finals,Wed,2024-05-01,Dortmund,NaN,Paris S-G,Signal Iduna Park,NaN
26,2023-2024,Semi-finals,Tue,2024-05-07,Paris S-G,NaN,Dortmund,Parc des Princes,NaN
27,2023-2024,Semi-finals,Wed,2024-05-08,Real Madrid,NaN,Bayern Munich,Estadio Santiago Bernabéu,NaN


In [45]:
# Preparar los datos de los próximos partidos para la predicción
features_upcoming = upcoming_matches[['Home', 'Away']]

# Añadir columna 'Referee' con un valor predeterminado si no está presente
if 'Referee' not in features_upcoming.columns:
    features_upcoming['Referee'] = 'Unknown'

# Preparar los datos de los próximos partidos para la predicción
X_upcoming = preprocessor.transform(features_upcoming)

# Hacer las predicciones con el modelo entrenado
predicted_winners = model.predict(X_upcoming)
predicted_winners = (predicted_winners > 0.5).astype(int)

# Asignar las predicciones al dataframe de próximos partidos
upcoming_matches['Predicted Winner'] = predicted_winners
upcoming_matches['Predicted Winner'] = upcoming_matches['Predicted Winner'].map({1: 'Home Wins', 0: 'Away Wins'})

# Mostrar los resultados
print(upcoming_matches[['Home', 'Away', 'Predicted Winner']])


1/1 [==============================] - 0s 99ms/step
             Home           Away Predicted Winner
24  Bayern Munich    Real Madrid        Away Wins
25       Dortmund      Paris S-G        Home Wins
26      Paris S-G       Dortmund        Home Wins
27    Real Madrid  Bayern Munich        Home Wins


C:\Users\albab\AppData\Local\Temp\ipykernel_22988\2519762111.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_upcoming['Referee'] = 'Unknown'
C:\Users\albab\AppData\Local\Temp\ipykernel_22988\2519762111.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_matches['Predicted Winner'] = predicted_winners
C:\Users\albab\AppData\Local\Temp\ipykernel_22988\2519762111.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [46]:
# Configurar los datos del partido final
final_match = pd.DataFrame({
    'Home': ['Paris S-G'],
    'Away': ['Bayern Munich'],
    'Referee': ['Unknown']  # Asumiendo que no conocemos el árbitro
})

# Preparar los datos para la predicción
X_final = preprocessor.transform(final_match)

# Hacer la predicción
predicted_final_winner = model.predict(X_final)
predicted_final_winner = (predicted_final_winner > 0.5).astype(int)

# Interpretar la predicción
final_winner = 'Home Wins' if predicted_final_winner else 'Away Wins'
print("Predicted Final Winner: ", 'Paris S-G' if final_winner == 'Home Wins' else ' Bayern Munich')


1/1 [==============================] - 0s 12ms/step
Predicted Final Winner:   Bayern Munich
